# Turn aggregated data into weekly_stats_stats_stats_stats_stats_stats-behavior data

In [1]:
import pandas as pd
import numpy as np

In [2]:
zoom = pd.read_excel(r"C:\Users\Admin\Documents\Studio Code\customer_retention_data\student-dropout-analysis\data_processed\final_zoom.xlsx", dtype={'SID':str})
zoom

,Topic,ID,Host name,Host email,Start time,End time,Participants,Duration (minutes),Name (original name),Email,...,Grade,SID_full,Name,EFL,Note,MVA/MVSM,concat,4 last digits,Học sinh cùng lớp có chung 4 số cuối ID,_merge
0,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,Mai Nguyen Truc Linh 01816,23001816@minhvietacademy.org,...,Grade 1,2300-1816,Mai Nguyễn Trúc Linh,EFL K5,Chính thức,MVA,MVA2300-1816,1816.0,NaN,both
1,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,Nguyen Pham Nhat Minh 02469,22002469@minhvietacademy.org,...,Grade 1,2200-2469,Nguyễn Phạm Nhật Minh,EFL K5,Chính thức,MVA,MVA2200-2469,2469.0,NaN,both
2,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,Yen Ha Tu Vy 02190,24002190@minhvietacademy.org,...,Grade 1,2400-2190,VY HÀ TÚ YÊN,EFL K5,Chính thức,MVA,MVA2400-2190,2190.0,NaN,both
3,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,Lai Ngan Khanh 01771,23001771@minhvietacademy.org,...,Grade 1,2300-1771,Lại Ngân Khánh,EFL K5,Chính thức,MVA,MVA2300-1771,1771.0,NaN,both
4,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,Tran Minh Khang 03927,23003927@minhvietacademy.org,...,Grade 1,2300-3927,Trần Minh Khang,EFL K5,Chính thức,MVA,MVA2300-3927,3927.0,NaN,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271177,25APBIO,942 5158 6913,Schedule 9,schedule9@minhvietacademy.org,5/5/2025 19:25,5/5/2025 20:23,47,58,"Nguyen Hung, Viet",NaN,...,Grade 9-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
271178,25APBIO,942 5158 6913,Schedule 9,schedule9@minhvietacademy.org,5/5/2025 19:25,5/5/2025 20:23,47,58,"Than Thu, Nguyen Minh",NaN,...,Grade 9-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
271179,25APBIO,942 5158 6913,Schedule 9,schedule9@minhvietacademy.org,5/5/2025 19:25,5/5/2025 20:23,47,58,"Nguyen Tien, Anh",NaN,...,Grade 9-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
271180,25APENG,996 9559 8473,Schedule 9,schedule9@minhvietacademy.org,5/5/2025 18:24,5/5/2025 19:25,40,61,phạm nguyên phúc hưng 02068,NaN,...,Grade 9-12,2400-2068,Phạm Nguyên Phúc Hưng,EFL 69,Chính thức,MVA,MVA2400-2068,2068.0,NaN,both


#### Aggregrate disconnected sessions
As students sometimes have internet issues and got disconnected from zoom, their participation is splitted into smaller sessions/records that seem like disengagement. So here we group them back together

In [4]:
#zoom events (ze)
ze = zoom.copy()

ze['Start time'] = pd.to_datetime(ze['Start time'], errors='coerce')
ze['End time'] = pd.to_datetime(ze['End time'], errors='coerce')
ze['Join time'] = pd.to_datetime(ze['Join time'], errors='coerce')
ze['Leave time'] = pd.to_datetime(ze['Leave time'], errors='coerce')


ze = (ze.groupby(['SID', 'Topic', 'Start time'])
              .agg(
                  end_time=('End time', 'max'),
                  student_duration=('student_stay_duration', 'sum'),
                  join_time=('Join time', 'min'),
                  leave_time=('Leave time', 'max'),
                  grade=('Grade', 'last'),
                  duration=('Duration (minutes)', 'first')
              )).reset_index()

ze['date'] = ze['Start time'].dt.floor("D")

ze

,SID,Topic,Start time,end_time,student_duration,join_time,leave_time,grade,duration,date
0,00000006,2506ELA,2025-02-24 18:24:00,2025-02-24 19:26:00,61,2025-02-24 18:24:00,2025-02-24 19:24:00,Grade 6,63,2025-02-24
1,00000006,2506ELA,2025-02-26 18:25:00,2025-02-26 19:26:00,45,2025-02-26 18:42:00,2025-02-26 19:26:00,Grade 6,62,2025-02-26
2,00000006,2506ELA,2025-03-03 18:26:00,2025-03-03 19:26:00,60,2025-03-03 18:27:00,2025-03-03 19:26:00,Grade 6,61,2025-03-03
3,00000006,2506ELA,2025-03-05 18:25:00,2025-03-05 19:27:00,9,2025-03-05 19:16:00,2025-03-05 19:25:00,Grade 6,63,2025-03-05
4,00000006,2506ELA,2025-03-17 18:27:00,2025-03-17 19:26:00,59,2025-03-17 18:27:00,2025-03-17 19:26:00,Grade 6,60,2025-03-17
...,...,...,...,...,...,...,...,...,...,...
207563,25000506,2501HUM,2025-05-27 19:25:00,2025-05-27 20:25:00,30,2025-05-27 19:55:00,2025-05-27 20:25:00,Grade 1,60,2025-05-27
207564,25000506,2501HUM,2025-05-29 19:27:00,2025-05-29 20:25:00,55,2025-05-29 19:30:00,2025-05-29 20:25:00,Grade 1,59,2025-05-29
207565,25000506,2501MAT,2025-05-14 19:26:00,2025-05-14 20:24:00,58,2025-05-14 19:26:00,2025-05-14 20:24:00,Grade 1,58,2025-05-14
207566,25000506,2501MAT,2025-05-21 19:26:00,2025-05-21 20:28:00,60,2025-05-21 19:28:00,2025-05-21 20:28:00,Grade 1,62,2025-05-21


#### Compute Minutes late / Minutes leaving early / Absent flag

In [38]:
grace_min = 5 #the school allows for 5 minutes grace

ze['minutes_late'] = ((ze['join_time'] - ze['Start time']).dt.total_seconds()/60) - grace_min
ze['minutes_late'] = ze['minutes_late'].clip(lower=0)

ze['minutes_left_early'] = ((ze['end_time'] - ze['leave_time']).dt.total_seconds()/60)
ze['minutes_left_early'] = ze['minutes_left_early'].clip(lower=0)

ze['attended_ratio'] = (ze['student_duration']/ze['duration'])
ze['very_short_flag'] = (ze['student_duration']<15).astype(int)
ze['partial_present'] = (ze['attended_ratio'] < 0.75).astype(int)
ze['is_present'] = (ze['attended_ratio']>0.25).astype(int)

ze.head()

,SID,Topic,Start time,end_time,student_duration,join_time,leave_time,grade,duration,date,minutes_late,minutes_left_early,attended_ratio,is_present,very_short_flag,partial_present
0,00000006,2506ELA,2025-02-24 18:24:00,2025-02-24 19:26:00,61,2025-02-24 18:24:00,2025-02-24 19:24:00,Grade 6,63,2025-02-24,0.0,2.0,0.968254,1,0,0
1,00000006,2506ELA,2025-02-26 18:25:00,2025-02-26 19:26:00,45,2025-02-26 18:42:00,2025-02-26 19:26:00,Grade 6,62,2025-02-26,12.0,0.0,0.725806,1,0,1
2,00000006,2506ELA,2025-03-03 18:26:00,2025-03-03 19:26:00,60,2025-03-03 18:27:00,2025-03-03 19:26:00,Grade 6,61,2025-03-03,0.0,0.0,0.983607,1,0,0
3,00000006,2506ELA,2025-03-05 18:25:00,2025-03-05 19:27:00,9,2025-03-05 19:16:00,2025-03-05 19:25:00,Grade 6,63,2025-03-05,46.0,2.0,0.142857,0,1,1
4,00000006,2506ELA,2025-03-17 18:27:00,2025-03-17 19:26:00,59,2025-03-17 18:27:00,2025-03-17 19:26:00,Grade 6,60,2025-03-17,0.0,0.0,0.983333,1,0,0


#### Aggregrate to day-level
As students can have multiple sessions per day, here we aggregrate to form the basis for week-level

In [40]:
day = (ze.groupby(['SID', 'date'])
       .agg(
           late_max=('minutes_late', 'max'), #max minutes late in that day
           late_mean=('minutes_late', 'mean'), #mean minutes late (over all the sessions in that day)
           late_sum=('minutes_late', 'sum'), #total minutes late in that day
           early_max=('minutes_left_early', 'max'), #max minutes left_early in that day
           early_mean=('minutes_left_early', 'mean'),#mean minutes left early (over all the sessions in that day)
           early_sum=('minutes_left_early', 'sum'), #total minutes left early in that day
           very_short_any=('very_short_flag', 'max'), #if there is 1 session attended under 15 minutes in that day or not 
           very_short_mean=('very_short_flag', 'mean'), #mean sessions atteneded under 15 minutes in that day
           very_short_sum=('very_short_flag', 'sum'), #total sessions atteneded under 15 minutes in that day
           partial_any=('partial_present', 'max'), #if there is 1 partially attended in that day or not 
           partial_mean=('partial_present', 'mean'), #mean sessions partially atteneded in that day
           partial_sum=('partial_present', 'sum'), #total sessions partially atteneded in that day
           attended_ratio_mean=('attended_ratio', 'mean'), #mean attended rate in that day
           attended_ratio_min=('attended_ratio', 'min'), #lowest attended rate in that day
           minutes_attended_day=('student_duration', 'sum'), #total minutes joined in that day
           total_duration_day=('duration', 'sum'), #total scheduled minutes of classes in that day
           is_present_sum=('is_present', 'sum')
       )).reset_index()

day['attended_ratio'] = day['minutes_attended_day']/day['total_duration_day'].replace(0,np.nan)
day.head()

,SID,date,late_max,late_mean,late_sum,early_max,early_mean,early_sum,very_short_any,very_short_mean,very_short_sum,partial_any,partial_mean,partial_sum,attended_ratio_mean,attended_ratio_min,minutes_attended_day,total_duration_day,is_present_sum,attended_ratio
0,00000006,2025-02-20,0.0,0.0,0.0,14.0,7.0,14.0,0,0.0,0,1,0.5,1,0.872881,0.745763,106,121,2,0.876033
1,00000006,2025-02-24,0.0,0.0,0.0,11.0,6.5,13.0,0,0.0,0,0,0.0,0,0.885766,0.803279,110,124,2,0.887097
2,00000006,2025-02-25,3.0,1.5,3.0,7.0,4.5,9.0,0,0.0,0,0,0.0,0,0.860656,0.836066,105,122,2,0.860656
3,00000006,2025-02-26,12.0,6.0,12.0,17.0,8.5,17.0,0,0.0,0,1,1.0,2,0.707166,0.688525,87,123,2,0.707317
4,00000006,2025-02-27,0.0,0.0,0.0,20.0,20.0,20.0,0,0.0,0,1,1.0,1,0.590164,0.590164,36,61,1,0.590164


#### Convert day -> week statistics

In [44]:
to_week = lambda d:d.to_period("W-MON").start_time
day['week_start'] = day['date'].map(to_week)

weekly_stats = (day.groupby(['SID', 'week_start'])
                .agg(
                    #Alert signs
                    late_max=('late_max', 'max'),
                    early_max=('late_max', 'max'),
                    total_late_minutes=('late_sum', 'sum'),
                    total_early_minutes=('early_sum', 'sum'),

                    #Trend
                    late_mean=('late_mean', 'mean'),
                    early_mean=('early_mean', 'mean'),
                    attended_ratio_mean=('attended_ratio_mean', 'mean'),
                    attended_ratio_week=('attended_ratio', 'mean'),


                    #Count the very short/partially-attend sessions
                    very_short_any=('very_short_any', 'max'),
                    total_very_short=('very_short_sum', 'sum'),
                    partial_any=('partial_any', 'max'),
                    total_partial=('partial_sum', 'sum'),

                    #others
                    days_present_w=('is_present_sum', 'sum'),
                    days_observed_w=('date', 'nunique'),

                    #general info
                    total_attended_minutes_w=('minutes_attended_day', 'sum'),
                    total_planned_minutes_w=('total_duration_day', 'sum')

                )
                ).reset_index()

weekly_stats

,SID,week_start,late_max,early_max,total_late_minutes,total_early_minutes,late_mean,early_mean,attended_ratio_mean,attended_ratio_week,very_short_any,total_very_short,partial_any,total_partial,days_present_w,days_observed_w,total_attended_minutes_w,total_planned_minutes_w
0,00000006,2025-02-18,0.0,0.0,0.0,27.0,0.000000,6.750,0.879324,0.881565,0,0,1,1,4,2,216,245
1,00000006,2025-02-25,12.0,12.0,15.0,77.0,1.875000,12.125,0.721873,0.721911,0,0,1,4,7,4,317,428
2,00000006,2025-03-04,46.0,46.0,87.0,108.0,14.500000,18.000,0.413700,0.412178,1,3,1,5,3,3,152,367
3,00000006,2025-03-11,0.0,0.0,0.0,48.0,0.000000,12.000,0.790847,0.789256,0,0,1,2,4,2,191,242
4,00000006,2025-03-18,0.0,0.0,0.0,100.0,0.000000,18.750,0.680633,0.682542,1,1,1,3,6,4,319,425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48098,25000396,2025-05-20,0.0,0.0,0.0,0.0,0.000000,0.000,0.966667,0.966667,0,0,0,0,3,3,174,180
48099,25000506,2025-05-06,0.0,0.0,0.0,0.0,0.000000,0.000,1.000000,1.000000,0,0,0,0,1,1,57,57
48100,25000506,2025-05-13,13.0,13.0,13.0,0.0,3.250000,0.000,0.915254,0.915254,0,0,1,1,4,4,215,235
48101,25000506,2025-05-20,6.0,6.0,9.0,0.0,3.000000,0.000,0.881903,0.881903,0,0,0,0,3,3,159,180


#### 4-week window (prediction frame)

In [35]:
HOLIDAYS = ['2024-09-02', '2024-09-03', '2024-11-05', '2024-11-11', '2024-11-28', '2024-11-29',
            '2024-12-23', '2024-12-24','2024-12-25','2024-12-26','2024-12-27','2024-12-28','2024-12-29','2024-12-30',
            '2024-12-31','2025-01-01','2025-01-02', '2025-01-20','2025-01-25','2025-01-26','2025-01-27','2025-01-28',
            '2025-01-29','2025-01-30','2025-01-31','2025-02-01','2025-02-02','2025-02-17','2025-04-05','2025-04-06',
            '2025-04-07','2025-04-30','2025-05-01','2025-05-26']

In [45]:
HOLIDAYS = pd.to_datetime(HOLIDAYS).date

def school_days_in_week_ex_holidays(monday_ts):
    # business days Mon..Sun, then filter to Mon–Fri and drop holidays
    bdays = pd.bdate_range(monday_ts, monday_ts + pd.Timedelta(days=6))  # business days in that calendar week
    return int(sum((d.weekday() <= 4) and (d.date() not in HOLIDAYS) for d in bdays))


weekly_stats["school_days_w"] = weekly_stats["week_start"].apply(school_days_in_week_ex_holidays).astype(int)

# Presence rate over intended school days (Mon–Fri minus holidays)
weekly_stats["present_rate_w"] = weekly_stats["days_present_w"] / weekly_stats["school_days_w"].replace(0, np.nan)

# Attendance ratio (time-based quality)
#weekly_stats["attendance_ratio_w"] = weekly_stats["total_attended_minutes_w"] / weekly_stats["total_planned_minutes_w"].replace(0, np.nan)

# --- Fill missing weeks (so weeks with no activity appear as explicit rows) ---
def fill_missing_weeks(df, sid_col="SID", week_col="week_start", freq="W-MON"):
    """
    Ensure every student has a continuous weekly index from their first to last seen week.
    - Takes a per-student subset `g`
    - Builds a full weekly date_range from min(week_start) to max(week_start)
    - Reindexes to insert missing Mondays as rows with NaNs
    - Keeps SID alongside
    Why: rolling windows then reflect *calendar* weeks; 0-activity weeks become real rows.
    """
    out = []
    for sid, g in df.groupby(sid_col):
        if g.empty:
            continue
        g = g.sort_values(week_col)
        full_idx = pd.date_range(g[week_col].min(), g[week_col].max(), freq=freq)
        gg = (g.set_index(week_col)
                .reindex(full_idx)                # insert missing weeks as NaN
                .rename_axis(week_col)
                .reset_index())
        gg[sid_col] = sid                         # carry SID down to the new rows
        out.append(gg)
    return pd.concat(out, ignore_index=True) if out else df.copy()

student_week = fill_missing_weeks(weekly_stats, sid_col="SID", week_col="week_start", freq="W-MON")

# Counts/minutes: fill NaN with 0; keep means/rates NaN when no denominator
for c in ["days_present_w","days_observed_w","total_attended_minutes_w","total_planned_minutes_w","school_days_w"]:
    if c in student_week.columns:
        student_week[c] = student_week[c].fillna(0)

# Optional: explicit inactivity flag (no presence and no minutes recorded that week)
student_week["inactive_week_flag"] = (
    (student_week.get("days_present_w", 0) == 0) &
    (student_week.get("total_attended_minutes_w", 0) == 0)
).astype(int)

# --- Rolling 4-week trend features (per student) ---
student_week = student_week.sort_values(["SID", "week_start"])

student_week


,week_start,SID,late_max,early_max,total_late_minutes,total_early_minutes,late_mean,early_mean,attended_ratio_mean,attended_ratio_week,...,total_very_short,partial_any,total_partial,days_present_w,days_observed_w,total_attended_minutes_w,total_planned_minutes_w,school_days_w,present_rate_w,inactive_week_flag
0,2025-02-24,00000006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,1
1,2025-03-03,00000006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,1
2,2025-03-10,00000006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,1
3,2025-03-17,00000006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,1
4,2025-03-24,00000006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51665,2025-05-12,25000396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,1
51666,2025-05-19,25000396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,1
51667,2025-05-12,25000506,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,1
51668,2025-05-19,25000506,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,1


In [46]:
student_week.to_csv('filled_missing_weeks.csv', index=False)

In [ ]:

for col in [
    "week_late_max_w","week_early_max_w",
    "avg_late_max_w","avg_early_max_w",
    "late_wmean_w","early_wmean_w",
    "very_short_any_w","partial_any_w",
    "present_rate_w","attendance_ratio_w"
]:
    if col in student_week.columns:
        student_week[col + "_4w"] = (student_week
            .groupby(SID_COL)[col]
            .rolling(ROLL_WEEKS, min_periods=1)
            .mean()
            .reset_index(level=0, drop=True))

# Ensure week_start is datetime
student_week["week_start"] = pd.to_datetime(student_week["week_start"])

print(">>> Sample of student×week Zoom features (Mon–Fri):")
print(student_week.head(12))

# Save
student_week.to_csv(OUTPUT_CSV, index=False)
print(f"\nSaved weekly zoom features to: {OUTPUT_CSV}")